In [ ]:
import concurrent.futures
from tqdm import tqdm

def process_iteration(qerr_i, latest_in, latest_tc, coll, pred_cumulate):
    # Your existing loop body here

# Your existing function definition here

def decodePreds_parallel(inputs, preds, truecards, collist):
    global global_table 
    global global_cols_alldomain 
    
    latest_inputs = inputs[:, -1, :]
    latest_truecards = truecards[:, -1]
    
    total_iterations = len(latest_inputs)
    q_error_list = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Use ThreadPoolExecutor for CPU-bound tasks; for IO-bound tasks, consider ProcessPoolExecutor
        futures = {executor.submit(process_iteration, qerr_i, latest_in, latest_tc, coll, pred_cumulate):
                   (qerr_i, latest_in, latest_tc, coll, pred_cumulate) for qerr_i, (latest_in, latest_tc, coll, pred_cumulate)
                   in enumerate(zip(latest_inputs, latest_truecards, collist, preds))}
        
        for future in tqdm(concurrent.futures.as_completed(futures), total=total_iterations, desc="Processing"):
            try:
                q_error_list.append(future.result())
            except Exception as e:
                print(f"An error occurred: {e}")
                
    return q_error_list
